## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.

Team - Car Crashes' Severity Prediction

## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [ ]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as Xet

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [ ]:
df.drop(columns='ID').describe()

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [ ]:
cols = ["date", "holiday"]
rows = []

# Parsing the XML file
xmlparse = Xet.parse('/kaggle/input/car-crashes-severity-prediction/holidays.xml')
root = xmlparse.getroot()
for i in root:
    date = i.find("date").text
    holiday = i.find("description").text

    rows.append({"date": date,
                "holiday": holiday})

df_holidays = pd.DataFrame(rows, columns=cols)


In [ ]:
df['timestamp'] =  pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
df['date'] = pd.to_datetime(df['timestamp'].dt.date, format='%Y-%m-%d') 

In [ ]:
df_new = pd.merge(df, df_holidays, how='left')

# Read Weather File

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df_weather = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df_weather.shape))


In [ ]:
df_date_weather = df_weather[['Year','Month','Day']].astype(str)
df_weather['date'] = df_date_weather.apply(lambda x: '-'.join(x.dropna().values.tolist()), axis=1)
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y-%m-%d') 

df_weather['Hour'] = pd.to_datetime(df_weather['Hour'], format='%H').dt.time
df_date_weather = df_weather[['date','Hour']].astype(str)
df_weather['date'] = df_date_weather.apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)


In [ ]:
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y-%m-%d %H') 


# Data Cleaning

In [ ]:
df_weather['Precipitation(in)'].fillna(0, inplace = True)
df_weather['Weather_Condition'].fillna(df_weather['Weather_Condition'].mode().iat[0], inplace = True)
df_weather['Temperature(F)'].fillna(df_weather['Temperature(F)'].median(), inplace=True)
df_weather['Humidity(%)'].fillna(df_weather['Humidity(%)'].median(), inplace=True)
df_weather['Visibility(mi)'].fillna(df_weather['Visibility(mi)'].median(), inplace=True)
df_weather["Wind_Speed(mph)"].fillna(np.random.choice(df_weather["Wind_Speed(mph)"][~df_weather["Wind_Speed(mph)"].isna()]),inplace = True)

In [ ]:
df_weather.drop(['Wind_Chill(F)'], axis=1, inplace=True) #  Wind_Chill(F) is identical to Temperature(F)

In [ ]:
df_weather.drop_duplicates(subset=['date'], inplace=True)

In [ ]:
df_new['date'] = df_new['timestamp'].dt.round('H')


# Merge 

In [ ]:
df_holiday_weather = pd.merge(df_new, df_weather, how='left')


In [ ]:
df_holiday_weather['holiday'][df_holiday_weather['holiday'].notna()] = True
df_holiday_weather['holiday'].fillna(False, inplace = True)

In [ ]:
df_holiday_weather['Side']=df_holiday_weather[['Side']].applymap(lambda  x: 1 if x =='R' else 0)

In [ ]:
df_holiday_weather_Sorted=df_holiday_weather.sort_values("timestamp")

In [ ]:
df_holiday_weather_Sorted.fillna(method="ffill" , axis=0 , inplace=True)

In [ ]:
df_holiday_weather_Sorted["year"]=df_holiday_weather_Sorted["timestamp"].dt.year
df_holiday_weather_Sorted["month"]=df_holiday_weather_Sorted["timestamp"].dt.month
df_holiday_weather_Sorted["day"]=df_holiday_weather_Sorted["timestamp"].dt.day
df_holiday_weather_Sorted["hour"]=df_holiday_weather_Sorted["timestamp"].dt.hour


In [ ]:
df_weather_condition=pd.get_dummies(df_holiday_weather_Sorted['Weather_Condition'])
df_holiday_weather_Sorted_Last=pd.concat([df_holiday_weather_Sorted, df_weather_condition], axis=1)
df_holiday_weather_Sorted_Last.shape

# Feature engineering

In [ ]:
# df_holiday_weather_Sorted_Last['hour']

In [ ]:
df_holiday_weather_Sorted_Last['hour_cat'] = pd.cut(df_holiday_weather_Sorted_Last['hour'], bins=4, labels=[0,1,2,3], ordered=False)

# df_holiday_weather_Sorted_Last['hour_cat'].head(50)

In [ ]:
df_holiday_weather_Sorted_Last['Severity'].value_counts()

# sampling

In [ ]:
df4 = df_holiday_weather_Sorted_Last[df_holiday_weather_Sorted_Last['Severity'] == 4]
df4

In [ ]:
df4 = df_holiday_weather_Sorted_Last[df_holiday_weather_Sorted_Last['Severity'] == 4]

# df4 = df4.append(new_row, ignore_index=True)
list1 = []
for _ in range(1000): 
    row_random = {}
    for i in df4.columns :
        row_random[i] = np.random.choice(df4[i].values)  
    list1.append(row_random)
df_4 = pd.DataFrame(list1)

In [ ]:
df_4['Severity'].value_counts()

In [ ]:
df_holiday_weather_Sorted_Last = pd.concat([df_holiday_weather_Sorted_Last, df_4], ignore_index=True)

In [ ]:
df1 = df_holiday_weather_Sorted_Last[df_holiday_weather_Sorted_Last['Severity'] == 1]

list1 = []
for _ in range(1000): 
    row_random = {}
    for i in df1.columns :
        row_random[i] = np.random.choice(df1[i].values)  
    list1.append(row_random)
df_1 = pd.DataFrame(list1)

In [ ]:
df_1.shape

In [ ]:
df_1['Severity'].value_counts()

In [ ]:
df_holiday_weather_Sorted_Last = pd.concat([df_holiday_weather_Sorted_Last, df_1], ignore_index=True)

In [ ]:
df_holiday_weather_Sorted_Last['Severity'].value_counts()

In [ ]:
# df_holiday_weather_Sorted_Last.tail(50)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_holiday_weather_Sorted_Last , test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID','Bump', 'Roundabout','Severity','timestamp','date' ,'Year', 'Day',
       'Month','Weather_Condition' , 'Selected', 'hour' ])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID','Bump', 'Roundabout','Severity','timestamp','date' ,'Year', 'Day',
       'Month','Weather_Condition' , 'Selected', 'hour'])
y_val = val_df['Severity']


In [ ]:
# X_train.shape

In [ ]:
# X_train.isnull().sum()

In [ ]:
X_train = X_train[['Stop','Lat', 'Lng', 'Distance(mi)','hour_cat']]

X_val = X_val[['Stop','Lat', 'Lng', 'Distance(mi)','hour_cat']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
# classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier = RandomForestClassifier(max_depth=20)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [ ]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [ ]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

In [ ]:
test_df['timestamp'] =  pd.to_datetime(test_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
test_df['date'] = pd.to_datetime(test_df['timestamp'].dt.date, format='%Y-%m-%d') 

In [ ]:
df_new = pd.merge(test_df, df_holidays, how='left')

In [ ]:
df_new['date'] = df_new['timestamp'].dt.round('H')


In [ ]:
# df_weather.isnull().sum()

In [ ]:
df_holiday_weather_test = pd.merge(df_new, df_weather, how='left')

In [ ]:
# df_holiday_weather_test.isnull().sum()

In [ ]:
df_holiday_weather_test['holiday'][df_holiday_weather_test['holiday'].notna()] = True
df_holiday_weather_test['holiday'].fillna(False, inplace = True)

In [ ]:
df_holiday_weather_test['Side']=df_holiday_weather_test[['Side']].applymap(lambda  x: 1 if x =='R' else 0)

In [ ]:
# df_holiday_weather_test.isnull().sum()

In [ ]:
df_holiday_weather_test_Sorted=df_holiday_weather_test.sort_values("timestamp")
df_holiday_weather_test_Sorted.fillna(method="ffill" , axis=0 , inplace=True)
df_holiday_weather_test_Sorted.fillna(method="bfill" , axis=0 , inplace=True)


In [ ]:
# df_holiday_weather_test_Sorted.isnull().sum()

In [ ]:
df_holiday_weather_test_Sorted["year"]=df_holiday_weather_test_Sorted["timestamp"].dt.year
df_holiday_weather_test_Sorted["month"]=df_holiday_weather_test_Sorted["timestamp"].dt.month
df_holiday_weather_test_Sorted["day"]=df_holiday_weather_test_Sorted["timestamp"].dt.day
df_holiday_weather_test_Sorted["hour"]=df_holiday_weather_test_Sorted["timestamp"].dt.hour

In [ ]:
df_weather_condition=pd.get_dummies(df_holiday_weather_test_Sorted['Weather_Condition'])
df_holiday_weather_test_Sorted_Last=pd.concat([df_holiday_weather_test_Sorted, df_weather_condition], axis=1)
df_holiday_weather_test_Sorted_Last.shape

In [ ]:
df_holiday_weather_test_Sorted_Last['hour_cat'] = pd.cut(df_holiday_weather_test_Sorted_Last['hour'], bins=4, labels=[0,1,2,3], ordered=False)


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [ ]:
X_test = df_holiday_weather_test_Sorted_Last.drop(columns=['hour', 'ID','Bump', 'Roundabout','timestamp','date' ,'Year', 'Day','Month','Weather_Condition' , 'Selected' ])

# You should update/remove the next line once you change the features used for training


X_test = X_test[['Stop','Lat', 'Lng', 'Distance(mi)','hour_cat']]
# X_test['Mist']=np.zeros(X_test.shape[0])
y_test_predicted = classifier.predict(X_test)
df_holiday_weather_test_Sorted_Last['Severity'] = y_test_predicted

df_holiday_weather_test_Sorted_Last.head()

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [ ]:
df_holiday_weather_test_Sorted_Last[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!